In [30]:
import soccerdata as sd
import pandas as pd
from tqdm import tqdm
import ScraperFC as sfc
import traceback

import warnings
import uuid

In [31]:
## Create a connection to Snowflake
from sqlalchemy import create_engine
import snowflake.connector

In [32]:
import snowflake.connector

In [59]:
from snowflake.connector.pandas_tools import write_pandas
from snowflake.sqlalchemy import URL

In [34]:
## Set options to view all columns
pd.set_option('display.max_columns', None)

In [6]:
scraper = sfc.FBRef()
try:
    # Scrape the table
    lg_table = scraper.scrape_league_table(year=2023, league='EPL')
except:
    # Catch and print any exceptions. This allows us to still close the
    # scraper below, even if an exception occurs.
    traceback.print_exc()
finally:
    # It's important to close the scraper when you're done with it. Otherwise,
    # you'll have a bunch of webdrivers open and running in the background.
    scraper.close()

Scraping 2023 EPL league table


d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\FBRef.py:216: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  lg_table = pd.read_html(str(lg_table_html))[0]


In [35]:
lg_table.sort_values(by='Squad', ascending=True, inplace=True)
lg_table.reset_index(drop=True, inplace=True)

In [37]:
fbref = sd.FBref(leagues="ENG-Premier League", seasons=["2223"])
fbref_team_standard = fbref.read_team_season_stats(stat_type='standard')
fbref_team_standard_oppo = fbref.read_team_season_stats(stat_type='standard', opponent_stats=True).reset_index()
fbref_team_shooting = fbref.read_team_season_stats(stat_type='shooting').reset_index()
fbref_team_passing = fbref.read_team_season_stats(stat_type='passing').reset_index()
fbref_player_defense = fbref.read_player_season_stats(stat_type='defense').reset_index()
fbref_team_defense = fbref.read_team_season_stats(stat_type='defense').reset_index()
fbref_team_possession = fbref.read_team_season_stats(stat_type='possession').reset_index()
fbref_team_misc = fbref.read_team_season_stats(stat_type='misc').reset_index()

[12/13/23 12:14:09] INFO     Saving cached data to C:\Users\ksbha\soccerdata\data\FBref               ]8;id=28154;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=261034;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py#91\91]8;;\

In [38]:
team_names = fbref_team_standard.reset_index().team
team_matches_played = fbref_team_standard['Playing Time'].MP
team_goals_scored = lg_table.GF
team_npxG_90 = fbref_team_standard['Per 90 Minutes'].npxG
team_goals_conceded = lg_table.GA
team_xG_against_90 = fbref_team_standard_oppo['Per 90 Minutes'].xG
team_shots = fbref_team_shooting['Standard'].Sh
team_shotsOT = fbref_team_shooting['Standard'].SoT
team_pass_completed = fbref_team_passing["Total"].Cmp
team_pass_attempted = fbref_team_passing["Total"].Att
team_tackles = fbref_team_defense.Tackles.Tkl
team_tackles_won = fbref_team_defense.Tackles.TklW
team_takeons_attempted = fbref_team_possession['Take-Ons'].Att
team_takeons_completed = fbref_team_possession['Take-Ons'].Succ
team_crossesintoPA = fbref_team_passing['CrsPA']
team_fouls_against = fbref_team_misc['Performance']['Fld']
team_fouls_made = fbref_team_misc['Performance']['Fls']
team_interceptions = fbref_team_defense['Int']
team_blocks_shots = fbref_team_defense['Blocks'].Sh
team_blocks_pass = fbref_team_defense['Blocks'].Pass
team_clearances = fbref_team_defense.Clr

In [50]:
team_names = fbref_team_standard.reset_index()[['team','url']]
team_names['TEAM_FBREF_ID'] = team_names['url'].apply(lambda x: x.split('/')[3])      ## Extracting team_id from URL
team_names.drop(columns=['url'], inplace=True)
team_names.columns = team_names.columns.droplevel(1)
team_names.rename(columns={'team': 'TEAM_NAME'}, inplace=True)

In [52]:
team_names['TEAM_FBREF_ID'] = team_names['TEAM_FBREF_ID'].astype('string')

In [62]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'FBREF_TEAMSTATS'

In [42]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

[12/13/23 12:14:26] INFO     Snowflake Connector for Python Version: 3.3.1, Python Version:       ]8;id=462376;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=776360;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#334\334]8;;\
                             3.11.5, Platform: Windows-10-10.0.22000-SP0                                           

                    INFO     This connection is in OCSP Fail Open Mode. TLS Certificates would   ]8;id=623802;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=594924;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1103\1103]8;;\
                             be checked for validity and revocation status. Any other                              
                             Certificate Revocation related exceptions or OCSP Responder                           
                             failures would be disregarded in favor of connectivity.                               

In [43]:
cur = conn.cursor()
cur.execute(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}")

[12/13/23 12:14:27] INFO     query: [USE WAREHOUSE FOOTY_STORE]                                       ]8;id=967791;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=434288;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

[12/13/23 12:14:28] INFO     query execution done                                                     ]8;id=305599;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=130779;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=418182;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=431895;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [44]:
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"
cur.execute(create_schema_sql)

[12/13/23 12:14:32] INFO     query: [CREATE SCHEMA IF NOT EXISTS FBREF_TEAMSTATS]                     ]8;id=685345;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=433113;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=209069;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=507090;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=262286;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=658525;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [45]:
print("The Snowflake warehouse being used is:", SNOWFLAKE_WAREHOUSE)

The Snowflake warehouse being used is: FOOTY_STORE


In [75]:
create_table_query = """
CREATE TABLE IF NOT EXISTS TEAMS (
	TEAM_NAME VARCHAR(255),
	TEAM_FBREF_ID VARCHAR(255) PRIMARY KEY
);
"""

In [76]:
cur.execute(create_table_query)

[12/13/23 12:30:51] INFO     query: [CREATE TABLE IF NOT EXISTS TEAMS ( TEAM_NAME VARCHAR(255),       ]8;id=430900;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=943615;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             TEAM_FBREF_ID VARCHAR...]                                                             

                    INFO     query execution done                                                     ]8;id=884971;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=285996;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=122154;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=64525;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [ ]:
def insert_to_snowflake_DB(primary_keys, table_name, dataframe):
    cur.execute(f"SELECT {', '.join(primary_keys)} FROM {table_name}")
    existing_ids = cur.fetchall()
    existing_ids = [id for id in existing_ids]
    dataframe.set_index(['TEAM_FBREF_ID', 'SEASON'], inplace=True)
    new_rows = dataframe[~dataframe.index.isin(existing_ids)]
    new_rows.reset_index(inplace=True)
    if not new_rows.empty:
        success, nchunks, nrows, _ = write_pandas(conn, new_rows, table_name)
        print(f"Inserted {nrows} new rows")
    else:
        print("No new rows to insert")

In [ ]:
insert_to_snowflake_DB(['TEAM_FBREF_ID'], 'TEAMS', team_names)

In [117]:
create_standard_team_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_STANDARD_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    MATCHES_PLAYED INT,
    PRIMARY KEY (TEAM_FBREF_ID, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.FBREF_TEAMSTATS.TEAMS(TEAM_FBREF_ID)
);
"""

In [118]:
cur.execute(create_standard_team_stats_query)

[12/13/23 12:47:51] INFO     query: [CREATE TABLE IF NOT EXISTS TEAM_STANDARD_STATS ( TEAM_FBREF_ID   ]8;id=578921;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=4855;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), SEA...]                                                                 

                    INFO     query execution done                                                     ]8;id=923141;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=703987;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=105403;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=566838;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [101]:
team_matches_played = team_matches_played.reset_index()[['team','season','MP']].rename(columns={'team': 'TEAM_NAME', 
                                                                            'season':'SEASON','MP': 'MATCHES_PLAYED'})

KeyError: "None of [Index(['team', 'season', 'MP'], dtype='object')] are in the [columns]"

In [91]:
## Merge team_matches_played with team_names to get TEAM_ID
team_matches_played = team_matches_played.merge(team_names, on='TEAM_NAME', 
                                                how='left')[['TEAM_FBREF_ID', 'MATCHES_PLAYED','SEASON']]

In [202]:
def insert_to_snowflake_DB(primary_keys, table_name, dataframe, replace=False):
    if replace:
        cur.execute(f"TRUNCATE TABLE {table_name}")
        success, nchunks, nrows, _ = write_pandas(conn, dataframe, table_name)
    else:
        cur.execute(f"SELECT {', '.join(primary_keys)} FROM {table_name}")
        existing_ids = cur.fetchall()
        existing_ids = [id for id in existing_ids]
        dataframe.set_index(['TEAM_FBREF_ID', 'SEASON'], inplace=True)
        new_rows = dataframe[~dataframe.index.isin(existing_ids)]
        new_rows.reset_index(inplace=True)
        dataframe.reset_index(inplace=True)
        if not new_rows.empty:
            success, nchunks, nrows, _ = write_pandas(conn, new_rows, table_name)
            print(f"Inserted {nrows} new rows")
        else:
            print("No new rows to insert")

In [163]:
insert_to_snowflake_DB(['TEAM_FBREF_ID', 'SEASON'], 'TEAM_STANDARD_STATS', team_matches_played)

[12/13/23 13:05:34] INFO     query: [SELECT TEAM_FBREF_ID, SEASON FROM TEAM_STANDARD_STATS]           ]8;id=594233;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=158126;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=567672;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=985965;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 0                                      ]8;id=901796;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=996468;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=43036;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=350946;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=992865;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=286850;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query execution done                                                     ]8;id=391795;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=463136;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/13/23 13:05:35] INFO     query: [COPY INTO "TEAM_STANDARD_STATS" /*                               ]8;id=32177;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=868063;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write...]                                     

[12/13/23 13:05:36] INFO     query execution done                                                     ]8;id=941211;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=907167;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=239320;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=827348;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Inserted 20 new rows


In [174]:
team_names

,TEAM_NAME,TEAM_FBREF_ID
0,Arsenal,18bb7c10
1,Aston Villa,8602292d
2,Bournemouth,4ba7cbea
3,Brentford,cd051869
4,Brighton,d07537b9
5,Chelsea,cff3d9bb
6,Crystal Palace,47c64c55
7,Everton,d3fd31cc
8,Fulham,fd962109
9,Leeds United,5bfb9659


In [164]:
create_team_attacking_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_ATTACKING_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    GOALS_SCORED INT,
    NPXG FLOAT,
    SHOTS INT,
    SHOTS_ON_TARGET INT,
    PASS_COMPLETED INT,
    PASS_ATTEMPTED INT,
    TAKEONS_ATTEMPTED INT,
    TAKEONS_COMPLETED INT,
    CROSSES_INTO_PA INT,
    FOULS_AGAINST INT,
    PRIMARY KEY (TEAM_FBREF_ID, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.FBREF_TEAMSTATS.TEAMS(TEAM_FBREF_ID)
);
"""

In [165]:
cur.execute(create_team_attacking_stats_query)

[12/13/23 13:11:41] INFO     query: [CREATE TABLE IF NOT EXISTS TEAM_ATTACKING_STATS ( TEAM_FBREF_ID  ]8;id=740009;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=284272;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), SE...]                                                                  

[12/13/23 13:11:42] INFO     query execution done                                                     ]8;id=385742;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=196648;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=631968;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=68842;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [166]:
team_attacking_stats = team_npxG_90.reset_index()[['team','season','npxG']].rename(columns={'team': 'TEAM_NAME', 
                                                            'season':'SEASON','npxG': 'NPXG'})

In [177]:
team_attacking_stats = team_attacking_stats.merge(team_names, on='TEAM_NAME').merge(team_matches_played, on=['TEAM_FBREF_ID', 'SEASON'])

In [179]:
team_attacking_stats.NPXG = team_attacking_stats.NPXG * team_attacking_stats.MATCHES_PLAYED

In [183]:
## Add team_goals_scored to team_attacking_stats as a new column with no merging on anything
team_attacking_stats['GOALS_SCORED'] = team_goals_scored

In [185]:
team_attacking_stats['SHOTS'] = team_shots
team_attacking_stats['SHOTS_ON_TARGET'] = team_shotsOT
team_attacking_stats['PASS_COMPLETED'] = team_pass_completed
team_attacking_stats['PASS_ATTEMPTED'] = team_pass_attempted
team_attacking_stats['TAKEONS_ATTEMPTED'] = team_takeons_attempted
team_attacking_stats['TAKEONS_COMPLETED'] = team_takeons_completed
team_attacking_stats['CROSSES_INTO_PA'] = team_crossesintoPA
team_attacking_stats['FOULS_AGAINST'] = team_fouls_against

In [187]:
team_attacking_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [192]:
team_attacking_stats.drop('MATCHES_PLAYED', axis=1, inplace=True)

In [203]:
insert_to_snowflake_DB(['TEAM_FBREF_ID', 'SEASON'], 'TEAM_ATTACKING_STATS', team_attacking_stats, replace=True)

[12/13/23 14:07:44] INFO     query: [TRUNCATE TABLE TEAM_ATTACKING_STATS]                             ]8;id=674488;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=113204;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

[12/13/23 14:07:45] INFO     query execution done                                                     ]8;id=929879;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=553614;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=606661;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=808358;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=929859;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=118954;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=533828;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=425830;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=770015;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=973429;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=956595;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=129021;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/13/23 14:07:46] INFO     query: [COPY INTO "TEAM_ATTACKING_STATS" /*                              ]8;id=967119;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=136084;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.writ...]                                      

[12/13/23 14:07:47] INFO     query execution done                                                     ]8;id=316579;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=421337;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=347072;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=687648;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [205]:
create_team_defending_stats_query = """
CREATE TABLE IF NOT EXISTS TEAM_DEFENDING_STATS (
    TEAM_FBREF_ID VARCHAR(255),
    SEASON INT,
    GOALS_CONCEDED INT,
    XG_AGAINST FLOAT,
    TACKLES INT,
    TACKLES_WON INT,
    FOULS_MADE INT,
    INTERCEPTIONS INT,
    BLOCKED_SHOTS INT,
    BLOCKED_PASSES INT,
    CLEARANCES INT,
    PRIMARY KEY (TEAM_FBREF_ID, SEASON),
    FOREIGN KEY (TEAM_FBREF_ID) REFERENCES GEGENSTATS.FBREF_TEAMSTATS.TEAMS(TEAM_FBREF_ID)
);
"""

In [206]:
cur.execute(create_team_defending_stats_query)

[12/13/23 14:10:26] INFO     query: [CREATE TABLE IF NOT EXISTS TEAM_DEFENDING_STATS ( TEAM_FBREF_ID  ]8;id=906799;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=42029;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             VARCHAR(255), SE...]                                                                  

[12/13/23 14:10:27] INFO     query execution done                                                     ]8;id=86011;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=595627;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=117605;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=397759;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [208]:
team_defending_stats = team_names.copy()

In [209]:
team_defending_stats['SEASON'] = team_attacking_stats.SEASON
team_defending_stats['GOALS_CONCEDED'] = team_goals_conceded
team_defending_stats['XG_AGAINST'] = team_xG_against_90*team_matches_played.MATCHES_PLAYED
team_defending_stats['TACKLES'] = team_tackles
team_defending_stats['TACKLES_WON'] = team_tackles_won
team_defending_stats['FOULS_MADE'] = team_fouls_made
team_defending_stats['INTERCEPTIONS'] = team_interceptions
team_defending_stats['BLOCKED_SHOTS'] = team_blocks_shots
team_defending_stats['BLOCKED_PASSES'] = team_blocks_pass
team_defending_stats['CLEARANCES'] = team_clearances

In [211]:
team_defending_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [212]:
insert_to_snowflake_DB(['TEAM_FBREF_ID', 'SEASON'], 'TEAM_DEFENDING_STATS', team_defending_stats)

[12/13/23 14:12:13] INFO     query: [SELECT TEAM_FBREF_ID, SEASON FROM TEAM_DEFENDING_STATS]          ]8;id=618522;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=431644;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=36243;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=345719;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 0                                      ]8;id=991124;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=55455;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=979246;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=235094;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

[12/13/23 14:12:14] INFO     query execution done                                                     ]8;id=921659;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=429977;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query execution done                                                     ]8;id=955475;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=227431;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

[12/13/23 14:12:15] INFO     query: [COPY INTO "TEAM_DEFENDING_STATS" /*                              ]8;id=301754;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=510352;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.writ...]                                      

[12/13/23 14:12:16] INFO     query execution done                                                     ]8;id=536714;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=619286;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=441110;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=867998;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Inserted 20 new rows
